In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
from torchvision import datasets, transforms
import torch.utils.data as data_utils
import scipy
import scipy.sparse

## getting train/valid/test data

In [7]:
data = np.load('dataset.npz', allow_pickle=True)
ingredients = data['ingredients']
recipes = data['recipes']
vectorized_len = np.vectorize(len)
recipes = recipes[vectorized_len(recipes) > 0]

def split_train_val_test(recipes, train=0.8, val=0.1, dataset_size=200000):
    shuffled = np.random.RandomState(0).permutation(recipes)
    n_train = int(len(shuffled) * train * dataset_size / len(recipes)) # * dataset_size / len(recipes) scale down
    n_val = int(len(shuffled) * val * dataset_size / len(recipes))
    return shuffled[:n_train], shuffled[n_train: n_train + n_val], shuffled[-n_val:]

train_recipes, val_recipes, test_recipes = split_train_val_test(recipes, dataset_size=100000)

In [8]:
train_recipes[:5]

array([array([ 219,  212,   46, 1133,  222,  657, 1343, 1014,   73,  140,   26,
          8,  286]),
       array([  77,  967, 3002,  199,    2,    3,   98,   18, 1477,   25,  895,
          1,  157,    0,    8]),
       array([  14,  134,  147,    3,   33,    9,   30, 3275,    1,   73,  511,
       1597]),
       array([198, 233,  14,   3,  33,  42, 120, 151,  10,   7,   1,   0,  21,
        26]),
       array([167,  52,  32,  13,   5, 224,  71,   0,  43,   4,  36])],
      dtype=object)

In [9]:
# we need to convert our data into one-hot encoding
# we'll also return the x vector (all - 1 ingredient) and y (missing ingredient)
def convert_one_hot(array):
    # here i'm getting an array of zeros
    # num rows is the size of the input array (ie how many recipes)
    # num cols is num of ingredients total (so we can 1-hot them)
#     one_hot = np.zeros((len(array), len(ingredients)))
#     inputs = np.zeros((len(array), len(ingredients)))
    inputs = scipy.sparse.dok_matrix((len(array), len(ingredients)), dtype=np.int)
    targets = np.empty(len(array))
    
    for i in range(len(array)):
        if len(array[i]) > 0:
            # this is just indexing into the ith row of the array (ith recipe)
            # and saying all the values in the recipe we're gonna set to 1
#             one_hot[i][array[i]] = 1
            
            # randomly choose one of the ingredients
            leave_out_idx = np.random.randint(len(array[i]))
            leave_out = array[i][leave_out_idx]
            leave_out_array = np.delete(array[i], leave_out_idx)
            for ingredient_index in leave_out_array:
                inputs[i, ingredient_index] = 1
            targets[i] = leave_out
            
        else:
            print("shouldn't get here ever")
        
#     return one_hot, inputs, targets

    inputs = scipy.sparse.csr_matrix(inputs)
    return inputs, targets

In [10]:
# csr to pytorch sparse
# thanks to https://gist.github.com/aesuli/319d71707a5ee96086aa2439b87d4e38

# import numpy as np
# from scipy.sparse import csr_matrix
# import torch

# __author__ = 'Andrea Esuli'

# Acsr = csr_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
# print('Acsr',Acsr)

# Acoo = Acsr.tocoo()
# print('Acoo',Acoo)

# Apt = torch.sparse.LongTensor(torch.LongTensor([Acoo.row.tolist(), Acoo.col.tolist()]),
#                               torch.LongTensor(Acoo.data.astype(np.int32)))
# print('Apt',Apt)


# last working code before sparse stuff
# mini_size = 10000
# x_mini = torch.tensor(train_x[:mini_size].astype(np.float32))
# y_mini = torch.tensor(train_y[:mini_size].astype(np.int_))
# mini_train_tensor = data_utils.TensorDataset(x_mini, y_mini)

In [11]:
def get_torch_tensor(data):
    x, y = convert_one_hot(data)
    x = x.tocoo()
    print(data.shape)
    x_tensor = torch.sparse_coo_tensor(torch.tensor([x.row.tolist(), x.col.tolist()]),
                                  torch.tensor(x.data.astype(np.float32)), 
                                   torch.Size([data.shape[0], len(ingredients)]))
    y_tensor = torch.tensor(y.astype(np.int_))
    tensor = data_utils.TensorDataset(x_tensor, y_tensor)
    return tensor
    

In [12]:
mini_size = 1000
mini_train_tensor = get_torch_tensor(train_recipes[:mini_size])
# mini_train_x, mini_train_y = convert_one_hot(train_recipes[:mini_size])
# mini_train_x = mini_train_x.tocoo()
# x_mini = torch.sparse.LongTensor(torch.LongTensor([mini_train_x.row.tolist(), mini_train_x.col.tolist()]),
#                               torch.LongTensor(mini_train_x.data.astype(np.int32)))
# y_mini = torch.LongTensor(mini_train_y)
# mini_train_tensor = data_utils.TensorDataset(x_mini, y_mini)

(1000,)


In [13]:
print(len(train_recipes))

80000


In [14]:
train_tensor = get_torch_tensor(train_recipes)
valid_tensor = get_torch_tensor(val_recipes)
test_tensor = get_torch_tensor(test_recipes)

# train_one_hot, train_x, train_y = convert_one_hot(train_recipes[:10])
# val_one_hot, val_x, val_y = convert_one_hot(val_recipes)
# test_one_hot, test_x, test_y = convert_one_hot(test_recipes)

# x_train = torch.tensor(train_x.astype(np.float32))
# y_train = torch.tensor(train_y.astype(np.int_))
# train_tensor = data_utils.TensorDataset(x_train, y_train) 

# x_valid = torch.tensor(val_x.astype(np.float32))
# y_valid = torch.tensor(val_y.astype(np.int_))
# valid_tensor = data_utils.TensorDataset(x_valid, y_valid) 

# x_test = torch.tensor(test_x.astype(np.float32))
# y_test = torch.tensor(test_y.astype(np.int_))
# test_tensor = data_utils.TensorDataset(x_test, y_test) 

(80000,)
(10000,)
(10000,)


In [15]:
# data stuff here takes some time to load so lets save in
# pickle so we can read later from there

with open("torch_tensors.pkl", "wb") as f:
    pkl.dump((train_tensor, valid_tensor, test_tensor), f)

In [16]:
# reading data

with open("torch_tensors.pkl", "rb") as f:
    train_tensor, valid_tensor, test_tensor = pkl.load(f)

## Neural Net

In [17]:
class Net(nn.Module):
    def __init__(self, params, num_ingredients):
        super(Net, self).__init__()
        self.params = params
        print(num_ingredients)
        self.fc0 = nn.Linear(num_ingredients, self.params["hidden_1"])
        self.fc1 = nn.Linear(self.params["hidden_1"], self.params["hidden_2"])
        self.fc2 = nn.Linear(self.params["hidden_2"], num_ingredients)

    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [58]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_num_correct = 0
    sum_loss = 0
    num_batches_since_log = 0
    
    losses = []

#     batches = tqdm(enumerate(train_loader), total=len(train_loader))
#     batches.set_description("Epoch NA: Loss (NA) Accuracy (NA %)")
#     for batch_idx, (data, target) in batches:
    for data, target in train_loader
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
#         print(data.shape)
        output = model(data)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(output, target)
#         loss = F.nll_loss(output, target)
        pred = output.max(1, keepdim=True)[1]
#         print(f"Prediction: {pred}, Actual: {target}, Loss: {loss}")
        correct = pred.eq(target.float().view_as(pred)).sum().item()
        sum_num_correct += correct
        sum_loss += loss.item()
        num_batches_since_log += 1
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
#         batches.set_description(
#           "Epoch {:d}: Loss ({:.2e},  Accuracy ({:02.0f}%)".format(
#             epoch, loss.item(), 100. * sum_num_correct / (num_batches_since_log * train_loader.batch_size))
#         )
        
    # return avg loss, accuracy of epoch
    return sum(losses)/len(losses), 100. * sum_num_correct / (num_batches_since_log * train_loader.batch_size)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # We use reduction = 'sum' here to ignore the impact of batch size and make 
            # this value comparable with the loss reported in the train loop above. Note,
            # though, that we divide by the len of the dataset below (so this is truly a per-element loss value)
#             test_loss += F.mse_loss(torch.clamp(output.view(target.shape), 0., 5.), target.float(), reduction='sum') # sum up the mean square loss
#             pred = torch.clamp(output, 0., 5.)
#             correct += pred.eq(target.float().view_as(pred)).sum().item()
            loss_fn = nn.CrossEntropyLoss()
            loss = torch.sum(loss_fn(output, target)).item() # sum up batch loss
            test_loss += loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
#     print('\nTest set: Average loss: {:.2e}\n'.format(test_loss))
    print('\nTest set: Average loss: {:.2e}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))

    # loss, accuracy
    return test_loss, test_accuracy

In [ ]:
args = dict()
args["seed"] = 1
args["no_cuda"] = False
args["log_interval"] = 50
args["batch_size"] = 256
args["test-batch-size"] = 1000

params = dict()
params["epochs"] = 10
params["lr"] = 0.1
params["hidden_1"] = len(ingredients)
params["hidden_2"] = len(ingredients)

use_cuda = not args["no_cuda"] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
mini_train_loader = torch.utils.data.DataLoader(
    mini_train_tensor,
    batch_size=args["batch_size"], shuffle=True, **kwargs)

In [48]:
train_loader = torch.utils.data.DataLoader(
    train_tensor,
    batch_size=args["batch_size"], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    test_tensor,
    batch_size=args["batch_size"], shuffle=True, **kwargs)

In [ ]:
model = Net(params, len(ingredients)).to(device)
optimizer = optim.Adam(model.parameters(), lr=params["lr"])

In [ ]:
#cross entropy loss, lr 0.01

torch.manual_seed(args["seed"])
for epoch in range(1, params["epochs"] + 1):
    train(model, device, train_loader, optimizer, epoch)
    
# crap i deleted the results lol but it was 32% accuracy after 10 epochs

In [41]:
# nll loss, lr 0.01

torch.manual_seed(args["seed"])
for epoch in range(1, params["epochs"] + 1):
    train(model, device, train_loader, optimizer, epoch)

<ipython-input-37-d695efa2b022>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  batches = tqdm(enumerate(train_loader), total=len(train_loader))


KeyboardInterrupt: 

In [50]:
# cross-entropy lr 0.001

torch.manual_seed(args["seed"])
for epoch in range(1, params["epochs"] + 1):
    train(model, device, train_loader, optimizer, epoch)

<ipython-input-42-7bc267542994>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  batches = tqdm(enumerate(train_loader), total=len(train_loader))


KeyboardInterrupt: 

In [ ]:
# cross-entropy lr 0.1

# torch.manual_seed(args["seed"])
# for epoch in range(1, params["epochs"] + 1):
#     train(model, device, train_loader, optimizer, epoch)

In [ ]:
# lol let's save the model in a pickle file

# cross entropy lr 0.001 rn
with open("nn_model.pkl", "wb") as f:
    pkl.dump(model, f)

In [51]:
with open("nn_model.pkl", "rb") as f:
    model = pkl.load(f)

In [59]:
test(model, device, test_loader)


Test set: Average loss: 2.65e-02, Accuracy: 1604/10000 (16%)



(0.026546246910095214, 16.04)